In [1]:
"""Collects data for file_properties_df.

file_properties_df holds metadata about data in our dataset, 
details of metadata can be seen at src/nna/tests/mock_data.py::mock_file_properties_df


"""


'Collects data for file_properties_df.\n\nfile_properties_df holds metadata about data in our dataset, \ndetails of metadata can be seen at src/nna/tests/mock_data.py::mock_file_properties_df\n\n\n'

In [35]:

from pathlib import Path
import pandas as pd
import datetime

from collections import Counter

from nna.fileUtils import list_files,getLength,read_file_properties_v2


In [13]:
# External Programs
# ffprobe version >= 4.3.1  
ffprobe_path = '/home/enis/sbin/ffprobe'

In [14]:
# PARAMETERS for new DATABASE

# increase version number accordinly 
previous_database_ver_str = 'V9'
new_database_ver_str = 'V10'

# where to save txt file storing length info
# old_data_folder="/home/enis/projects/nna/data/"
data_folder = '/scratch/enis/data/nna/database/'
#/scratch/enis/data/nna/database

# path to search for audio files
# where
# ignore_folders=['/tank/data/nna/real/stinchcomb/']

search_path="/tank/data/nna/real/"

ignore_folders=["/tank/data/nna/real/stinchcomb/dups/",
                "/tank/data/nna/real/stinchcomb/excerpts/",
                "/tank/data/nna/real/stinchcomb/"]
# search_path="/tank/data/nna/real/stinchcomb/"



In [15]:
# # PARAMETERS for external or small inferences

# # PARAMETERS  

# # increase version number accordinly 
# previous_database_ver_str = ''
# new_database_ver_str = 'V1'

# # where to save txt file storing length info
# # old_data_folder="/home/enis/projects/nna/data/"
# # data_folder = '/scratch/enis/data/nna/database/'
# data_folder = '/scratch/enis/data/nna/collar_database/'

# #/scratch/enis/data/nna/database

# # path to search for audio files
# # where
# # ignore_folders=['/tank/data/nna/real/stinchcomb/']

# # search_path="/tank/data/nna/real/"
# search_path = '/tank/data/nna/audio_collars/'

# ignore_folders=["/tank/data/nna/real/stinchcomb/dups/","/tank/data/nna/real/stinchcomb/excerpts/"]
# # search_path="/tank/data/nna/real/stinchcomb/"



In [19]:
# create Relative Path names

# if we already have a list of files we can load them 
# files_list_path=data_folder+"stinchcomb_files_pathV1.txt"
files_list_path=data_folder+ f"allFields_path{new_database_ver_str}.txt"

# if we calculated audio lengths and saved them into text file, 
# we can load them
fileswlen_path = data_folder+ f"allFields_wlen_f{new_database_ver_str}.txt"
filesWError_out = data_folder+f"allFields_wERROR_f{new_database_ver_str}.txt"

# do NOT add pkl extension at the end
pkl_file_name=f"allFields_data{new_database_ver_str}"


# this is the current info we have so we can check if we already processed a file before
current_pkl_file = data_folder + f"allFields_data{previous_database_ver_str}.pkl"

In [20]:
current_pkl_file

'/scratch/enis/data/nna/database/allFields_dataV9.pkl'

In [21]:
# Find files
# in given search path ignoring given directories
if Path(fileswlen_path).exists():
    with open(files_list_path,"r") as f:
        lines=f.readlines()
        files_path_list=[line.strip() for line in lines]
else:
    files_path_list=list_files(search_path,ignore_folders)



In [22]:
print('example file:',files_path_list[0])
print('total number of files:',len(files_path_list))
assert Path(files_path_list[0]).is_dir()==False


example file: /tank/data/nna/real/anwr/31/2019/S4A10297_20190504_000000.flac
total number of files: 255714


In [23]:
# count file extensions
suffix2files={}

files_suffixes=[]
for m in files_path_list:
    m=Path(m)
    mSuffix = m.suffix.lower()
    
    files_suffixes.append(mSuffix)
    suffix2files.setdefault(mSuffix,[]).append(m)

        
print(Counter(files_suffixes))


Counter({'.flac': 255714})


In [24]:
#V4: Counter({'.flac': 112053, '.aac': 9101, '.mp3': 388, '.flac~': 1})
#V5: Counter({'.flac': 151527, '.aac': 9101, '.mp3': 388, '.flac~': 1})
#V6: Counter({'.flac': 165976, '.aac': 9101, '.mp3': 388, '.flac~': 1, '.txt': 1})
#V7: Counter({'.flac': 204692, '.aac': 9101, '.wav': 2340, '.mp3': 388, '.txt': 3, '.flac~': 1, '.filepart': 1})

#V8: Counter({'.flac': 227227, '.aac': 9101, '.mp3': 388, '.flac~': 1, '.filepart': 1, '.txt': 1})

#V10: Counter({'.flac': 259798})


In [25]:
# 
# Load previous data
if Path(current_pkl_file+'non').exists() :

    current_file_properties_df=pd.read_pickle(str(current_pkl_file))

    # remove files we already know about
    currentFileSet = set(current_file_properties_df.index)
    currentFileSet = set([str(m) for m in currentFileSet])
    foundFileSet = set(files_path_list)
    foundFileSet = foundFileSet.difference(currentFileSet)
    New_files_path_list = list(foundFileSet)
    a_str="new",len(New_files_path_list),"previously",len(currentFileSet),"total",len(files_path_list)
    print(a_str)
else:
    print('does NOT exists',current_pkl_file)
    New_files_path_list = files_path_list[:]


does NOT exists /scratch/enis/data/nna/database/allFields_dataV9.pkl


In [2]:
# # Load or calculate Audio length
import pickle


filesWError = []

length_dict = {}
c=0
for f in New_files_path_list:
    if c % 1000 == 0:
        print(c/len(New_files_path_list))
    c+=1
    length=float(getLength(f))
    length_dict[f]=length


fileswlen=length_dict

# with open('fileswlen.pickle', 'wb') as handle:
#     pickle.dump(fileswlen, handle, protocol=pickle.HIGHEST_PROTOCOL)


# with open('fileswlen.pickle', 'rb') as handle:
#     fileswlen = pickle.load(handle)


    

In [42]:
fileswlen_filtered={}
for f,length in fileswlen.items():
    if 'dalton/09/2021/9/' in f:
        continue
    else:
        fileswlen_filtered[f]=length
fileswlen = fileswlen_filtered

In [43]:
filesWError=[]
for f,length in fileswlen.items():
    if length==-1:
        filesWError.append(f)
    

assert len(filesWError)==0

In [35]:
# print and save files with errors

print(len(filesWError))
with open(filesWError_out,"w") as f:
    for line in filesWError:
        join_lines = [str(i) for i in line]
        f.write(",".join(join_lines)+"\n")

0


In [33]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
# file_properties
file_properties,exceptions = read_file_properties_v2(list(fileswlen.keys()),debug=0)


In [45]:
set([tuple(str(ex).split("/")[5:7]) for ex in exceptions])

set()

In [47]:
len(exceptions),len(New_files_path_list)

(0, 255714)

In [60]:
for f,lengthSeconds in fileswlen.items():
    if file_properties.get(Path(f)) is not None:
        file_properties[Path(f)]["durationSec"] = lengthSeconds
        file_properties[Path(f)]["timestampEnd"] = file_properties[Path(f)]["timestamp"] + datetime.timedelta(seconds=lengthSeconds)
    else:
        print("file not found",f)
file_properties_df=pd.DataFrame(file_properties).T


In [72]:
assert (file_properties_df[file_properties_df['durationSec']>0]).shape==file_properties_df.shape
## ignore files with 0 length
# file_properties_df = file_properties_df[file_properties_df['durationSec']>0]

In [61]:
## these files are too short, I should investiage these
# (file_properties_df[file_properties_df['durationSec']<21])

In [63]:
# merge with previous file properties
merged_file_properties_df = pd.concat([file_properties_df,current_file_properties_df])


In [65]:
merged_file_properties_df.to_pickle(data_folder+pkl_file_name+".pkl")

In [80]:
merged_file_properties_df.to_pickle(data_folder+pkl_file_name+"2.pkl")

## check dataset stats

In [81]:
pd.__version__

'1.2.0'

In [66]:
current_file_properties_df=pd.read_pickle(data_folder+pkl_file_name+".pkl")

In [70]:
print('total days of recording',((sum(current_file_properties_df['durationSec'])/3600)/24))

total days of recording 3536.5799098982093


In [71]:
import numpy as np
files_df_2019=current_file_properties_df[current_file_properties_df['year']=='2019']
site_region=list({(val[1],val[0]) for val in  (files_df_2019[['locationId','region']].values)})
site_region.sort()

```bash

cat '/scratch/enis/data/nna/flow_tracking/region-location-list_all-fields_datav10_without-stinchcomb.txt' | \
            parallel --csv -P 20 -n 1 -q python clipping_cli.py \
            --region {1} --location {2} --output_folder \
            '/scratch/enis/data/nna/clipping_info/all-merged_2021-10-28/' \
            --file_database '/scratch/enis/data/nna/database/allFields_dataV10.pkl' >>logs_2021-10-28.txt 2>&1 \
            && /scratch/enis/conda/envs/speechEnv/bin/python \
	/home/enis/projects/nna/src/nna/slack_message.py \
	 -t "cpu job ended" -m 'clipping ended' \
	|| /scratch/enis/conda/envs/speechEnv/bin/python \
	/home/enis/projects/nna/src/nna/slack_message.py \
	 -t "cpu job FAILED" -m 'clipping FAILED'

```

In [63]:
for region,site_id in site_region:
    site_files_df_2019=files_df_2019[files_df_2019['site_id']==site_id]
    region_files_df_2019=site_files_df_2019[site_files_df_2019['region']==region]
    t=region_files_df_2019['timestamp'].sort_values()[0]
    print(site_id,region,':',t.month,t.day,'(M/D)')
    # print(region_files_df_2019['timestamp'].sort_values()[0])

31 anwr : 5 4 (M/D)
32 anwr : 5 4 (M/D)
33 anwr : 5 4 (M/D)
34 anwr : 5 5 (M/D)
35 anwr : 5 5 (M/D)
36 anwr : 5 5 (M/D)
37 anwr : 5 5 (M/D)
38 anwr : 5 4 (M/D)
39 anwr : 5 4 (M/D)
40 anwr : 5 4 (M/D)
41 anwr : 5 5 (M/D)
42 anwr : 5 5 (M/D)
43 anwr : 5 5 (M/D)
44 anwr : 5 5 (M/D)
45 anwr : 5 5 (M/D)
46 anwr : 5 4 (M/D)
47 anwr : 5 5 (M/D)
48 anwr : 5 5 (M/D)
49 anwr : 5 5 (M/D)
50 anwr : 5 5 (M/D)
01 dalton : 3 16 (M/D)
02 dalton : 3 16 (M/D)
03 dalton : 3 16 (M/D)
04 dalton : 3 16 (M/D)
05 dalton : 3 16 (M/D)
06 dalton : 3 17 (M/D)
07 dalton : 3 17 (M/D)
08 dalton : 3 17 (M/D)
09 dalton : 3 17 (M/D)
10 dalton : 3 18 (M/D)
11 dempster : 7 20 (M/D)
12 dempster : 3 23 (M/D)
13 dempster : 3 23 (M/D)
14 dempster : 3 23 (M/D)
15 dempster : 3 23 (M/D)
16 dempster : 3 23 (M/D)
17 dempster : 3 24 (M/D)
19 dempster : 3 24 (M/D)
20 dempster : 3 24 (M/D)
21 dempster : 3 24 (M/D)
22 dempster : 3 26 (M/D)
23 dempster : 3 26 (M/D)
24 dempster : 3 26 (M/D)
25 dempster : 3 26 (M/D)
AR01 ivvavik : 5 2 (

(2019, 5, 4)